---
title: Lab 3 - Mason Brehmer
echo: true
format: 
  html: 
    embed-resources: true
    code-fold: true
    toc: true

---
[View the GitHub Repository](https://github.com/masonbrehmer/GSB_544_Labs/tree/main)

In [49]:
import pandas as pd
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")
xmas2 = pd.read_csv("https://www.dropbox.com/scl/fi/p9x9k8xwuzs9rhp582vfy/xmas_2.csv?rlkey=kvc3j3lmyn4opcidsrhcmrof1&dl=1")


In [50]:
xmas.info()
xmas2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Day           12 non-null     int64 
 1   Day.in.Words  12 non-null     object
 2   Gift.Item     12 non-null     object
 3   Verb          7 non-null      object
 4   Adjective     4 non-null      object
 5   Location      1 non-null      object
dtypes: int64(1), object(5)
memory usage: 708.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Day           12 non-null     int64 
 1   Day.in.Words  12 non-null     object
 2   Gift.Item     12 non-null     object
 3   Verb          7 non-null      object
 4   Adjective     4 non-null      object
 5   Location      1 non-null      object
dtypes: int64(1), object(5)
memory usage: 708.0+ bytes


# Function 1 and Unit Tests

In [51]:
# # This was my initial code for the first function but it needed to be vectorized so I commented it out
# def pluralize_gift(gift):
#   """
#   Returns plural of a noun

#   Parameters
#   ----------
#   gift: str
#     A noun

#   Return
#   ------
#   str
#     Plural version
#   """
# # Implemented the help code snippets into the skeleton code.
# # However the code using if statements is not vectorized
#   if gift.find("oo") >= 0:
#     gift = gift.replace("oo", "ee")
#   elif gift.endswith("y") >= 0:
#     gift = gift[:-1] + "ies"
#   else:
#     gift = gift + "s"

#   return gift

In [52]:
#| code-fold: false
#Vectorized funciton 1
def pluralize_gift_vec(gifts):
    plural = gifts + "s"

    # Replace when word ends with "y"
    # Initially attempted the using .where but output wasn't showing so I asked chat if there was another function and it gave me .mask.
    plural = plural.mask(gifts.str.endswith("y", na=False),
                         gifts.str[:-1] + "ies")

    # Replace when word contains "oo"
    plural = plural.mask(gifts.str.contains("oo", na=False),
                         gifts.str.replace("oo", "ee", regex=False))

    return plural


In [53]:
# Asked chat to build two unit tests
pluralize_gift_vec(pd.Series(["goose"]))


0    geese
dtype: object

In [54]:
print(pluralize_gift_vec(pd.Series(["goose", "lady", "ring", "lebron", ])))

0      geese
1     ladies
2      rings
3    lebrons
dtype: object


In [55]:
# Will work if your function is vectorized!
pluralize_gift_vec(xmas['Gift.Item'])

0     partridges
1          doves
2           hens
3          birds
4          rings
5          geese
6          swans
7          maids
8         ladies
9          lords
10        pipers
11      drummers
Name: Gift.Item, dtype: object

# Function 2 and Unit Tests

In [56]:
#| code-fold: false
def make_phrase(num, num_word, item, verb, adjective, location):
   """
   Returns a phrase for a day of Christmas

   Parameters
  ----------
   num : int
    Day number
   num_word : str
    Day number in words
   item : str
    Gift item
   verb : str
    Gift verb
   adjective : str
    Gift adjective
   location : str
    Gift location

   Return
   ------
   str
     Phrase
   """

   # Step 1: Replace NAs with blank strings
   verb = verb.fillna("")
   adjective = adjective.fillna("")
   location = location.fillna("")

   # Step 2: Pluralize items when day > 1
   item_plural = item.mask(num > 1, pluralize_gift_vec(item))

   # Step 3: Checking if item starts with a vowel
   starts_with_vowel = item.str[0].str.lower().isin(list("aeiou"))

   # Step 4: Determine the "day word" to use
   # If it's the first day, we use "a" or "an"
   #Chat suggested I create a copy to make it easier to work with
   day_word = num_word.copy()
   #if it starts with a vowel
   day_word.loc[num == 1] = day_word.loc[num == 1].mask(
       starts_with_vowel.loc[num == 1], "an"
   )
   #If it does not start with a vowel
   day_word.loc[num == 1] = day_word.loc[num == 1].mask(
       ~starts_with_vowel.loc[num == 1], "a"
   )

   # Step 5: Assembling the phrase
   phrase = (
       day_word + " " +
       adjective + " " +
       item_plural + " " +
       verb + " " +
       location
   )

   # I asked chat once I finished if there was any ways to make the output look cleaner and it output suggestion code with .strip to clean up spaces
   phrase = phrase.str.replace(r"\s+", " ", regex=True).str.strip()

   return phrase

In [57]:
# Mapping words to ensure our phrasing is correct and says two not second
mapping_words = {
    2: "two", 3: "three", 4: "four",
    5: "five", 6: "six", 7: "seven", 8: "eight",
    9: "nine", 10: "ten", 11: "eleven", 12: "twelve"
}

xmas["Day_in_words"] = xmas["Day"].map(mapping_words)
xmas2["Day_in_words"] = xmas2["Day"].map(mapping_words)


In [58]:
# Adding the extra column with the full phrase to our xmas dataframe to get ready for the final step
xmas["Full.Phrase"] = make_phrase(
    num = xmas["Day"],
    num_word = xmas["Day_in_words"],
    item = xmas["Gift.Item"],
    verb = xmas["Verb"],
    adjective = xmas["Adjective"],
    location = xmas["Location"]
)
xmas


,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location,Day_in_words,Full.Phrase
0,1,first,partridge,NaN,NaN,in a pear tree,NaN,a partridge in a pear tree
1,2,second,dove,NaN,turtle,NaN,two,two turtle doves
2,3,third,hen,NaN,french,NaN,three,three french hens
3,4,fourth,bird,NaN,calling,NaN,four,four calling birds
4,5,fifth,ring,NaN,golden,NaN,five,five golden rings
5,6,sixth,goose,a-laying,NaN,NaN,six,six geese a-laying
6,7,seventh,swan,a-swimming,NaN,NaN,seven,seven swans a-swimming
7,8,eighth,maid,a-milking,NaN,NaN,eight,eight maids a-milking
8,9,ninth,lady,dancing,NaN,NaN,nine,nine ladies dancing
9,10,tenth,lord,a-leaping,NaN,NaN,ten,ten lords a-leaping


In [59]:
#Adding the extra column with full phrase to our xmas2 dataframe to get ready for the final step
xmas2["Full.Phrase"] = make_phrase(
    num = xmas2["Day"],
    num_word = xmas2["Day_in_words"],
    item = xmas2["Gift.Item"],
    verb = xmas2["Verb"],
    adjective = xmas2["Adjective"],
    location = xmas2["Location"]
)
xmas2


,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location,Day_in_words,Full.Phrase
0,1,first,email,NaN,NaN,from Cal Poly,NaN,an email from Cal Poly
1,2,second,point,NaN,meal,NaN,two,two meal points
2,3,third,pen,NaN,lost,NaN,three,three lost pens
3,4,fourth,review,NaN,course,NaN,four,four course reviews
4,5,fifth,exam,NaN,practice,NaN,five,five practice exams
5,6,sixth,grader,grading,NaN,NaN,six,six graders grading
6,7,seventh,senior,stressing,NaN,NaN,seven,seven seniors stressing
7,8,eighth,mom,a-calling,NaN,NaN,eight,eight moms a-calling
8,9,ninth,party,bumping,NaN,NaN,nine,nine parties bumping
9,10,tenth,load,of laundry,NaN,NaN,ten,ten loads of laundry


In [60]:
# Asked chat to create a mock data unit test
df = pd.DataFrame({
    "num": [1, 5, 6],
    "num_word": ["one", "five", "six"],
    "item": ["artridge", "ring", "goose"],
    "verb": ["a-singing", "a-shining", "a-laying"],
    "adjective": ["", "golden", ""],
    "location": ["in a pear tree", "by the fire", "on the lake"]
})

# Test Step 2 output
make_phrase(df["num"], df["num_word"], df["item"], df["verb"], df["adjective"], df["location"])

0       an artridge a-singing in a pear tree
1    five golden rings a-shining by the fire
2             six geese a-laying on the lake
dtype: object

In [61]:
#Testing to see if it works on the xmas dataset
xmas[["Day", "Full.Phrase"]].head(12)


,Day,Full.Phrase
0,1,a partridge in a pear tree
1,2,two turtle doves
2,3,three french hens
3,4,four calling birds
4,5,five golden rings
5,6,six geese a-laying
6,7,seven swans a-swimming
7,8,eight maids a-milking
8,9,nine ladies dancing
9,10,ten lords a-leaping


In [62]:
#Testing to see if it works on the xmas2 dataset
xmas2[["Day", "Full.Phrase"]].head(12)

,Day,Full.Phrase
0,1,an email from Cal Poly
1,2,two meal points
2,3,three lost pens
3,4,four course reviews
4,5,five practice exams
5,6,six graders grading
6,7,seven seniors stressing
7,8,eight moms a-calling
8,9,nine parties bumping
9,10,ten loads of laundry


# Function 3 and Final output

In [63]:
#| code-fold: false
def sing_day(dataset, num, phrase_col):
  """
  Returns a string with a song for days of Christmas
  
  Parameters
  ----------
  dataset : pandas dataframe
    Dataset with the song lyrics
  num : int
    Day number
  phrase_col : str
    Column name with the phrase
    
  Return
  ------
  str
    Song for the given day
 
  """
  
  # Step 1: Setup the intro line
  # mapping to make sure that the intro has first, second, third etc.
  number_word_map = {
        1: "first", 2: "second", 3: "third", 4: "fourth",
        5: "fifth", 6: "sixth", 7: "seventh", 8: "eighth",
        9: "ninth", 10: "tenth", 11: "eleventh", 12: "twelfth"
    }

  num_word = number_word_map[num]
  intro = "On the " + num_word + " day of Christmas, my true love sent to me:"
  
  # Step 2: Sing the gift phrases
  # Hint: What order are they gifts sung in each day
  gifts = ""
  # Looping backwards from the current day down
  for i in range(num, 0, -1):
    #Locating the phrase in hte dataset that matches the current day
    line = dataset.loc[dataset["Day"] == i, phrase_col].values[0]
    if i == 1 and num != 1:
      # Adds and if we are on the first day and it was not the first verse for the output. 
        gifts += "and " + line + "."
    else:
        gifts += line + "\n"

    # Step 4: Combine intro and verses , strip removes the whitespace or linebreaks at the end of the string. 
  song = intro + "\n" + gifts.strip()
  return song


In [64]:
for day in range(1, 13):
  print(sing_day(xmas, day, "Full.Phrase"))
  # Colab suggested the seperator between days to make it more readable.
  print("-" * 20)

On the first day of Christmas, my true love sent to me:
a partridge in a pear tree
--------------------
On the second day of Christmas, my true love sent to me:
two turtle doves
and a partridge in a pear tree.
--------------------
On the third day of Christmas, my true love sent to me:
three french hens
two turtle doves
and a partridge in a pear tree.
--------------------
On the fourth day of Christmas, my true love sent to me:
four calling birds
three french hens
two turtle doves
and a partridge in a pear tree.
--------------------
On the fifth day of Christmas, my true love sent to me:
five golden rings
four calling birds
three french hens
two turtle doves
and a partridge in a pear tree.
--------------------
On the sixth day of Christmas, my true love sent to me:
six geese a-laying
five golden rings
four calling birds
three french hens
two turtle doves
and a partridge in a pear tree.
--------------------
On the seventh day of Christmas, my true love sent to me:
seven swans a-swimming

In [65]:
for day in range(1, 13):
  print(sing_day(xmas2, day, "Full.Phrase"))
  # Colab suggested the seperator between days to make it more readable.
  print("-" * 20)

On the first day of Christmas, my true love sent to me:
an email from Cal Poly
--------------------
On the second day of Christmas, my true love sent to me:
two meal points
and an email from Cal Poly.
--------------------
On the third day of Christmas, my true love sent to me:
three lost pens
two meal points
and an email from Cal Poly.
--------------------
On the fourth day of Christmas, my true love sent to me:
four course reviews
three lost pens
two meal points
and an email from Cal Poly.
--------------------
On the fifth day of Christmas, my true love sent to me:
five practice exams
four course reviews
three lost pens
two meal points
and an email from Cal Poly.
--------------------
On the sixth day of Christmas, my true love sent to me:
six graders grading
five practice exams
four course reviews
three lost pens
two meal points
and an email from Cal Poly.
--------------------
On the seventh day of Christmas, my true love sent to me:
seven seniors stressing
six graders grading
five pr

In [66]:
print(sing_day(xmas, 12, "Full.Phrase"))


On the twelfth day of Christmas, my true love sent to me:
twelve drummers drumming
eleven pipers piping
ten lords a-leaping
nine ladies dancing
eight maids a-milking
seven swans a-swimming
six geese a-laying
five golden rings
four calling birds
three french hens
two turtle doves
and a partridge in a pear tree.


In [67]:
print(sing_day(xmas2, 12, "Full.Phrase"))

On the twelfth day of Christmas, my true love sent to me:
twelve hours sleeping
eleven friends goodbye-ing
ten loads of laundry
nine parties bumping
eight moms a-calling
seven seniors stressing
six graders grading
five practice exams
four course reviews
three lost pens
two meal points
and an email from Cal Poly.


In [68]:
print(sing_day(xmas2, 8, "Full.Phrase"))


On the eighth day of Christmas, my true love sent to me:
eight moms a-calling
seven seniors stressing
six graders grading
five practice exams
four course reviews
three lost pens
two meal points
and an email from Cal Poly.


In [69]:
print(sing_day(xmas, 1, "Full.Phrase"))


On the first day of Christmas, my true love sent to me:
a partridge in a pear tree
